<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

This notebook should be run using with Python 3.x with Spark runtime environment. If you are viewing this in Watson Studio and do not see Python 3.x with Spark in the upper right corner of your screen, please update the runtime now. It requires service credentials for the following services:

<li>Watson OpenScale
<li>Watson Machine Learning
<li>DB2

## Credentials for IBM Cloud Pak for Data
In the following code boxes, you need to replace the sample data with your own credentials. You can acquire the information from your system administrator or through the Cloud Pak for Data dashboard.

### Obtaining your Watson OpenScale credentials
You can retrieve the URL by running the following command: `oc get route -n namespace1 --no-headers | awk '{print $2}'`
Replace the `namespace1` variable with your namespace.

You should have been assigned a username and password when you were added to the Cloud Pak for Data system. You might need to ask either your database administrator or your system administrator for some of the information.

In [ ]:
WOS_CREDENTIALS = "YOUR_WOS_CREDENTIALS_HERE"

#####################################
# Example credentials will look below
#####################################

# WOS_CREDENTIALS = {
#     "url": "https://CLUSTER_NAME",
#     "username": "admin",
#     "password": "password"
# }

### Your Watson OpenScale GUID
For most systems, the default GUID is already entered for you. You would only need to update this particular entry if the GUID was changed from the default.

In [ ]:
WOS_GUID="00000000-0000-0000-0000-000000000000"

### Your Watson OpenScale GUID
For most systems, the default GUID is already entered for you. You would only need to update this particular entry if the GUID was changed from the default.

In [ ]:
WML_CREDENTIALS = WOS_CREDENTIALS.copy()
WML_CREDENTIALS['instance_id']='openshift'
WML_CREDENTIALS['version']='2.5.0'

### Your database credentials and schema
Normally, you must obtain the database credentials and schema name from your database administrator, however, if you deployed one of the database options from the Cloud Pak for Data UI, you should be able to retrieve credentials yourself by going to My data and right-clicking on the database tile.

In [ ]:
DATABASE_CREDENTIALS = "YOUR_DB_CREDENTIALS_HERE"

#####################################
# Example credentials will look below
#####################################

# DATABASE_CREDENTIALS = {
#     "db": "SAMPLE",
#     "db_type": "db2",
#     "hostname": "hostname",
#     "password": "password",
#     "port": 50000,
#     "username": "db2inst1"
# }


In [ ]:
SCHEMA_NAME = "YOUR_SCHEMA_NAME_HERE"

## Integration system credentials (IBM OpenPages MRG) 
As part of the closed beta, you were sent an email message from IBM that provides the URL, username, and password of an IBM OpenPages system that you can use. Replace the following credentials with the ones that you received in that email message.

In [ ]:
OPENPAGES_CREDENTIALS = "OPENPAGES_CREDENTIALS_HERE"

#####################################
# Example credentials will look below
#####################################

# OPENPAGES_CREDENTIALS = {
#    "url": "https://softlayer-dev1.op-ibm.com",
#    "username": "username",
#    "password": "password"
# }

## Package installation
The following opensource packages must be installed into this notebook instance so that they are available to use during processing.

In [ ]:
!rm -rf /home/spark/shared/user-libs/python3.6*

!pip install pyspark==2.3 --no-cache | tail -n 1
!pip install numpy==1.15.4 --no-cache | tail -n 1
!pip install --upgrade ibm-ai-openscale --no-cache | tail -n 1
!pip install --upgrade SciPy --no-cache | tail -n 1

## Load the training data from Github
So you don't have to manually generate training data, we've provided a sample and placed it in a publicly available Github repo.

In [ ]:
!rm german_credit_data_biased_training.csv
!wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/german_credit_data_biased_training.csv

## Create an OpenPages Model

The following cells creates and OpenPages GRC Object (Model) that we will use as an integrated system reference to which we will publish the OpenScale metrics generated as a part of the MRM risk evaluations. 

In [ ]:
import json
import requests
import base64
from requests.auth import HTTPBasicAuth
import time

In [ ]:
headers={}
headers["Content-Type"] = "application/json"
headers["Accept"] = "application/json"
auth = HTTPBasicAuth(OPENPAGES_CREDENTIALS["username"], OPENPAGES_CREDENTIALS["password"])

GET_OPENPAGES_MODEL_DEFINITION_URL = OPENPAGES_CREDENTIALS["url"] + "/grc/api/types/Model"

response = requests.get(GET_OPENPAGES_MODEL_DEFINITION_URL, headers=headers, auth=auth)
json_data = response.json()

if "id" in json_data:
    openpages_model_definition_id = json_data["id"]

if "fieldDefinitions" in json_data:
    for fieldDefinition in json_data["fieldDefinitions"]["fieldDefinition"]:
        if fieldDefinition["name"] == "MRG-Model:Model Owner":
            model_owner_field_definition_id = fieldDefinition["id"]
        elif fieldDefinition["name"] == "MRG-Model:Model Non Model":
            model_non_model_field_definition_id = fieldDefinition["id"]
        elif fieldDefinition["name"] == "MRG-Model:Machine Learning Model":
            machine_learning_model_field_definition_id = fieldDefinition["id"]
        elif fieldDefinition["name"] == "MRG-Model:OpenScale Monitor":
            openscale_monitor_field_definition_id = fieldDefinition["id"]
        elif fieldDefinition["name"] == "Description":
            description_field_definition_id = fieldDefinition["id"]
        elif fieldDefinition["name"] == "MRG-Model:Status":
            model_status_field_definition_id = fieldDefinition["id"]
        elif fieldDefinition["name"] == "MRG-Model:Candidate Status":
            model_candidate_status_field_definition_id = fieldDefinition["id"]

In [ ]:
headers={}
headers["Content-Type"] = "application/json"
headers["Accept"] = "application/json"
auth = HTTPBasicAuth(OPENPAGES_CREDENTIALS["username"], OPENPAGES_CREDENTIALS["password"])

payload = {
  "description": "OpenPages model created by the MRM E2E notebook for Credit Risk Model",
  "fields": {
    "field": [
      {
        "id": model_owner_field_definition_id,
        "name": "MRG-Model:Model Owner",
        "dataType": "STRING_TYPE",
        "value": OPENPAGES_CREDENTIALS["username"]
      },
      {
        "id": description_field_definition_id,
        "name": "Description",
        "dataType": "STRING_TYPE",
        "value": "OpenPages model for Watson OpenScale MRM Demo"
      },
      {
        "id": model_non_model_field_definition_id,
        "name": "MRG-Model:Model Non Model",
        "dataType": "ENUM_TYPE",
        "enumValue": {
          "name": "Model"
        }
      },
      {
        "id": machine_learning_model_field_definition_id,
        "name": "MRG-Model:Machine Learning Model",
        "dataType": "ENUM_TYPE",
        "enumValue": {
          "name": "Yes"
        }
      },
      {
        "id": openscale_monitor_field_definition_id,
        "name": "MRG-Model:OpenScale Monitor",
        "dataType": "ENUM_TYPE",
        "enumValue": {
          "name": "Yes"
        }
      },
      {
        "id": model_status_field_definition_id,
        "name": "MRG-Model:Model Status",
        "dataType": "ENUM_TYPE",
        "enumValue": {
          "name": "Under Development"
        }
      },
      {
        "id": model_candidate_status_field_definition_id,
        "name": "MRG-Model:Candidate Status",
        "dataType": "ENUM_TYPE",
        "enumValue": {
          "name": "Confirmed"
        }
      }
    ]
  },
  "typeDefinitionId": openpages_model_definition_id
}

CREATE_OPENPAGES_MODEL_URL = OPENPAGES_CREDENTIALS["url"] + "/grc/api/contents"
response = requests.post(CREATE_OPENPAGES_MODEL_URL, json=payload, headers=headers, auth=auth)
json_data = response.json()

openpages_model_name=json_data["name"]
openpages_model_id=json_data["id"]

print("OpenPages Model Name = {}".format(openpages_model_name))
print("OpenPages Model Id = {}".format(openpages_model_id))
print("OpenPages URL to view the model = {0}{1}{2}".format(OPENPAGES_CREDENTIALS["url"], "/app/jspview/react/grc/task-view/", openpages_model_id))

# Deploy the models

The following cells deploy both the pre-prod and challenger models into the Watson Machine Learning instance.

In [ ]:
PRE_PROD_MODEL_NAME="German Credit Risk Model - PreProd"
PRE_PROD_DEPLOYMENT_NAME="German Credit Risk Model - PreProd"

PRE_PROD_CHALLENGER_MODEL_NAME="German Credit Risk Model - Challenger"
PRE_PROD_CHALLENGER_DEPLOYMENT_NAME="German Credit Risk Model - Challenger"

PRE_PROD_SPACE_NAME="pre-prod"

## Initialize WML client and set the space_id parameter
Analytic spaces are used to categorize assets and can be associated with a project inside of Watson Studio.

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
wml_client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)
print(wml_client.service_instance.get_url())

wml_client.spaces.list()

# Find and set the default space
space_name=PRE_PROD_SPACE_NAME
spaces = wml_client.spaces.get_details()['resources']
space_id = None
for space in spaces:
    if space['entity']['name'] == space_name:
        space_id = space["metadata"]["guid"]
if space_id is None:
    space_id = wml_client.spaces.store(
        meta_props={wml_client.spaces.ConfigurationMetaNames.NAME: space_name})["metadata"]["guid"]
wml_client.set.default_space(space_id)

## Deploy the Spark Credit Risk Model to WML

The following cell deploys the Spark version of the German Credit Risk Model to the specified Machine Learning instance in the specified deployment space. You'll notice that this version of the German Credit Risk model has an auc-roc score around 71%.

In [ ]:
import numpy 
numpy.version.version

import pandas as pd
import json

from pyspark import SparkContext, SQLContext
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier,GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import StringIndexer, VectorAssembler, IndexToString
from pyspark.sql.types import StructType, DoubleType, StringType, ArrayType

from pyspark.sql import SparkSession
from pyspark import SparkFiles

spark = SparkSession.builder.getOrCreate()
pd_data = pd.read_csv("german_credit_data_biased_training.csv", sep=",", header=0)
spark_df = spark.read.csv(path="german_credit_data_biased_training.csv", sep=",", header=True, inferSchema=True)
spark_df.head()

(train_data, test_data) = spark_df.randomSplit([0.9, 0.1], 24)
print("Number of records for training: " + str(train_data.count()))
print("Number of records for evaluation: " + str(test_data.count()))

si_CheckingStatus = StringIndexer(inputCol='CheckingStatus', outputCol='CheckingStatus_IX')
si_CreditHistory = StringIndexer(inputCol='CreditHistory', outputCol='CreditHistory_IX')
si_LoanPurpose = StringIndexer(inputCol='LoanPurpose', outputCol='LoanPurpose_IX')
si_ExistingSavings = StringIndexer(inputCol='ExistingSavings', outputCol='ExistingSavings_IX')
si_EmploymentDuration = StringIndexer(inputCol='EmploymentDuration', outputCol='EmploymentDuration_IX')
si_Sex = StringIndexer(inputCol='Sex', outputCol='Sex_IX')
si_OthersOnLoan = StringIndexer(inputCol='OthersOnLoan', outputCol='OthersOnLoan_IX')
si_OwnsProperty = StringIndexer(inputCol='OwnsProperty', outputCol='OwnsProperty_IX')
si_InstallmentPlans = StringIndexer(inputCol='InstallmentPlans', outputCol='InstallmentPlans_IX')
si_Housing = StringIndexer(inputCol='Housing', outputCol='Housing_IX')
si_Job = StringIndexer(inputCol='Job', outputCol='Job_IX')
si_Telephone = StringIndexer(inputCol='Telephone', outputCol='Telephone_IX')
si_ForeignWorker = StringIndexer(inputCol='ForeignWorker', outputCol='ForeignWorker_IX')
si_Label = StringIndexer(inputCol="Risk", outputCol="label").fit(spark_df)
label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=si_Label.labels)

va_features = VectorAssembler(
inputCols=["CheckingStatus_IX", "CreditHistory_IX", "LoanPurpose_IX", "ExistingSavings_IX",
           "EmploymentDuration_IX", "Sex_IX", "OthersOnLoan_IX", "OwnsProperty_IX", "InstallmentPlans_IX",
           "Housing_IX", "Job_IX", "Telephone_IX", "ForeignWorker_IX", "LoanDuration", "LoanAmount",
           "InstallmentPercent", "CurrentResidenceDuration", "LoanDuration", "Age", "ExistingCreditsCount",
           "Dependents"], outputCol="features")

classifier=GBTClassifier(featuresCol="features")

pipeline = Pipeline(
stages=[si_CheckingStatus, si_CreditHistory, si_EmploymentDuration, si_ExistingSavings, si_ForeignWorker,
        si_Housing, si_InstallmentPlans, si_Job, si_LoanPurpose, si_OthersOnLoan,
        si_OwnsProperty, si_Sex, si_Telephone, si_Label, va_features, classifier, label_converter])

model = pipeline.fit(train_data)
predictions = model.transform(test_data)
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction")
auc = evaluator.evaluate(predictions)

print("Accuracy = %g" % auc)

# Remove existing model and deployment
MODEL_NAME=PRE_PROD_MODEL_NAME
DEPLOYMENT_NAME=PRE_PROD_DEPLOYMENT_NAME

deployment_details = wml_client.deployments.get_details()
for deployment in deployment_details['resources']:
    deployment_id = deployment['metadata']['guid']
    model_id = deployment['entity']['asset']['href'].split('/')[3].split('?')[0]
    if deployment['entity']['name'] == DEPLOYMENT_NAME:
        print('Deleting deployment id', deployment_id)
        wml_client.deployments.delete(deployment_id)
        print('Deleting model id', model_id)
        wml_client.repository.delete(model_id)
wml_client.repository.list_models()

# Save Model
model_props_rf = {
    wml_client.repository.ModelMetaNames.NAME: MODEL_NAME,
    wml_client.repository.ModelMetaNames.DESCRIPTION: MODEL_NAME,
    wml_client.repository.ModelMetaNames.RUNTIME_UID: "spark-mllib_2.3",
    wml_client.repository.ModelMetaNames.TYPE: "mllib_2.3"
}

published_model_details = wml_client.repository.store_model(model=model, meta_props=model_props_rf, training_data=train_data, pipeline=pipeline)
print(published_model_details)

# List models in the repository
wml_client.repository.list_models()

# Get the model UID
pre_prod_model_uid = wml_client.repository.get_model_uid(published_model_details)
pre_prod_model_uid


# Deploy model
wml_deployments = wml_client.deployments.get_details()
pre_prod_deployment_uid = None
for deployment in wml_deployments['resources']:
    if DEPLOYMENT_NAME == deployment['entity']['name']:
        pre_prod_deployment_uid = deployment['metadata']['guid']
        break

if pre_prod_deployment_uid is None:
    print("Deploying model...")
    meta_props = {
        wml_client.deployments.ConfigurationMetaNames.NAME: DEPLOYMENT_NAME,
        wml_client.deployments.ConfigurationMetaNames.DESCRIPTION: DEPLOYMENT_NAME,
        wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
    deployment = wml_client.deployments.create(artifact_uid=pre_prod_model_uid, name=DEPLOYMENT_NAME, meta_props=meta_props)
    pre_prod_deployment_uid = wml_client.deployments.get_uid(deployment)

print("Model id: {}".format(pre_prod_model_uid))
print("Deployment id: {}".format(pre_prod_deployment_uid))

pre_prod_deployment_uid=wml_client.deployments.get_uid(deployment)
pre_prod_deployment_uid

fields = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"]
values = [
  ["no_checking",13,"credits_paid_to_date","car_new",1343,"100_to_500","1_to_4",2,"female","none",3,"savings_insurance",46,"none","own",2,"skilled",1,"none","yes"],
  ["no_checking",24,"prior_payments_delayed","furniture",4567,"500_to_1000","1_to_4",4,"male","none",4,"savings_insurance",36,"none","free",2,"management_self-employed",1,"none","yes"],
  ["0_to_200",26,"all_credits_paid_back","car_new",863,"less_100","less_1",2,"female","co-applicant",2,"real_estate",38,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",14,"no_credits","car_new",2368,"less_100","1_to_4",3,"female","none",3,"real_estate",29,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",4,"no_credits","car_new",250,"less_100","unemployed",2,"female","none",3,"real_estate",23,"none","rent",1,"management_self-employed",1,"none","yes"],
  ["no_checking",17,"credits_paid_to_date","car_new",832,"100_to_500","1_to_4",2,"male","none",2,"real_estate",42,"none","own",1,"skilled",1,"none","yes"],
  ["no_checking",33,"outstanding_credit","appliances",5696,"unknown","greater_7",4,"male","co-applicant",4,"unknown",54,"none","free",2,"skilled",1,"yes","yes"],
  ["0_to_200",13,"prior_payments_delayed","retraining",1375,"100_to_500","4_to_7",3,"male","none",3,"real_estate",37,"none","own",2,"management_self-employed",1,"none","yes"]
]

payload_scoring = {"fields": fields,"values": values}
payload = {
    wml_client.deployments.ScoringMetaNames.INPUT_DATA: [payload_scoring]
}
scoring_response = wml_client.deployments.score(pre_prod_deployment_uid, payload)

scoring_response

## Deploy the Scikit-Learn Credit Risk Model to Watson Machine Learning

The following cell deploys the Scikit-learn version of the German Credit Risk Model to the specified Machine Learning instance in the specified deployment space. This version of the German Credit Risk model has an auc-roc score around 85% and will be called the "Challenger."

In [ ]:
import pandas as pd
import json
import sys
import numpy
import sklearn
import sklearn.ensemble
numpy.set_printoptions(threshold=sys.maxsize)
from sklearn.utils.multiclass import type_of_target
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_validate
from sklearn.metrics import get_scorer
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report

data_df=pd.read_csv ("german_credit_data_biased_training.csv")

data_df.head()

target_label_name = "Risk"
feature_cols= data_df.drop(columns=[target_label_name])
label= data_df[target_label_name]

# Set model evaluation properties
optimization_metric = 'roc_auc'
random_state = 33
cv_num_folds = 3
holdout_fraction = 0.1

if type_of_target(label.values) in ['multiclass', 'binary']:
    X_train, X_holdout, y_train, y_holdout = train_test_split(feature_cols, label, test_size=holdout_fraction, random_state=random_state, stratify=label.values)
else:
    X_train, X_holdout, y_train, y_holdout = train_test_split(feature_cols, label, test_size=holdout_fraction, random_state=random_state)

# Data preprocessing transformer generation

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('OrdinalEncoder', OrdinalEncoder(categories='auto',dtype=numpy.float64 ))])

numeric_features = feature_cols.select_dtypes(include=['int64', 'float64']).columns
categorical_features = feature_cols.select_dtypes(include=['object']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Initiate model and create pipeline
model=sklearn.ensemble.gradient_boosting.GradientBoostingClassifier()
gbt_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])
model_gbt=gbt_pipeline.fit(X_train, y_train)

y_pred = model_gbt.predict(X_holdout)


# Evaluate model performance on test data and Cross validation
scorer = get_scorer(optimization_metric)
scorer(model_gbt,X_holdout, y_holdout)

# Cross validation -3 folds
cv_results = cross_validate(model_gbt,X_train,y_train, scoring={optimization_metric:scorer})
numpy.mean(cv_results['test_' + optimization_metric])

print(classification_report(y_pred, y_holdout))


# Remove existing model and deployment
MODEL_NAME=PRE_PROD_CHALLENGER_MODEL_NAME
DEPLOYMENT_NAME=PRE_PROD_CHALLENGER_DEPLOYMENT_NAME

deployment_details = wml_client.deployments.get_details()
for deployment in deployment_details['resources']:
    deployment_id = deployment['metadata']['guid']
    model_id = deployment['entity']['asset']['href'].split('/')[3].split('?')[0]
    if deployment['entity']['name'] == DEPLOYMENT_NAME:
        print('Deleting deployment id', deployment_id)
        wml_client.deployments.delete(deployment_id)
        print('Deleting model id', model_id)
        wml_client.repository.delete(model_id)
wml_client.repository.list_models()

# Store Model
model_props_gbt = {
    wml_client.repository.ModelMetaNames.NAME: MODEL_NAME,
    wml_client.repository.ModelMetaNames.DESCRIPTION: MODEL_NAME,
    wml_client.repository.ModelMetaNames.RUNTIME_UID: "scikit-learn_0.20-py3.6",
    wml_client.repository.ModelMetaNames.TYPE: "scikit-learn_0.20"
}

published_model_details = wml_client.repository.store_model(model=model_gbt, meta_props=model_props_gbt, training_data=feature_cols,training_target=label)
print(published_model_details)

# List models in the repository
wml_client.repository.list_models()

# Get the model UID
challenger_model_uid = wml_client.repository.get_model_uid(published_model_details)
challenger_model_uid


# Deploy model
wml_deployments = wml_client.deployments.get_details()
challenger_deployment_uid = None
for deployment in wml_deployments['resources']:
    if DEPLOYMENT_NAME == deployment['entity']['name']:
        challenger_deployment_uid = deployment['metadata']['guid']
        break

if challenger_deployment_uid is None:
    print("Deploying model...")
    meta_props = {
        wml_client.deployments.ConfigurationMetaNames.NAME: DEPLOYMENT_NAME,
        wml_client.deployments.ConfigurationMetaNames.DESCRIPTION: DEPLOYMENT_NAME,
        wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
    deployment = wml_client.deployments.create(artifact_uid=challenger_model_uid, meta_props=meta_props)
    challenger_deployment_uid = wml_client.deployments.get_uid(deployment)

print("Model id: {}".format(challenger_model_uid))
print("Deployment id: {}".format(challenger_deployment_uid))

challenger_deployment_uid=wml_client.deployments.get_uid(deployment)
challenger_deployment_uid


# Sample scoring
fields = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"]
values = [
  ["no_checking",13,"credits_paid_to_date","car_new",1343,"100_to_500","1_to_4",2,"female","none",3,"savings_insurance",46,"none","own",2,"skilled",1,"none","yes"],
  ["no_checking",24,"prior_payments_delayed","furniture",4567,"500_to_1000","1_to_4",4,"male","none",4,"savings_insurance",36,"none","free",2,"management_self-employed",1,"none","yes"],
  ["0_to_200",26,"all_credits_paid_back","car_new",863,"less_100","less_1",2,"female","co-applicant",2,"real_estate",38,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",14,"no_credits","car_new",2368,"less_100","1_to_4",3,"female","none",3,"real_estate",29,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",4,"no_credits","car_new",250,"less_100","unemployed",2,"female","none",3,"real_estate",23,"none","rent",1,"management_self-employed",1,"none","yes"],
  ["no_checking",17,"credits_paid_to_date","car_new",832,"100_to_500","1_to_4",2,"male","none",2,"real_estate",42,"none","own",1,"skilled",1,"none","yes"],
  ["no_checking",33,"outstanding_credit","appliances",5696,"unknown","greater_7",4,"male","co-applicant",4,"unknown",54,"none","free",2,"skilled",1,"yes","yes"],
  ["0_to_200",13,"prior_payments_delayed","retraining",1375,"100_to_500","4_to_7",3,"male","none",3,"real_estate",37,"none","own",2,"management_self-employed",1,"none","yes"]
]

payload_scoring = {"fields": fields,"values": values}
payload = {
    wml_client.deployments.ScoringMetaNames.INPUT_DATA: [payload_scoring]
}
scoring_response = wml_client.deployments.score(challenger_deployment_uid, payload)

scoring_response


# Configure OpenScale 
The notebook will now import the necessary libraries and set up a Python OpenScale client.

In [ ]:
from ibm_ai_openscale import APIClient4ICP
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *
from ibm_ai_openscale.supporting_classes import PayloadRecord, Feature
from ibm_ai_openscale.supporting_classes.enums import *

In [ ]:
ai_client = APIClient4ICP(aios_credentials=WOS_CREDENTIALS)
ai_client.version

## Create schema and datamart

### Set up datamart
Watson OpenScale uses a database to store payload logs and calculated metrics. If an OpenScale datamart exists in Db2, the existing datamart will be used and no data will be overwritten.

In [ ]:
try:
    data_mart_details = ai_client.data_mart.get_details()
    print('Using existing external datamart')
except:
    print('Setting up external datamart')
    ai_client.data_mart.setup(db_credentials=DATABASE_CREDENTIALS, schema=SCHEMA_NAME)

In [ ]:
data_mart_details = ai_client.data_mart.get_details()

In [ ]:
data_mart_details

## Generate an ICP token

The following is a function that will generate an ICP access token used to interact with the Watson OpenScale APIs

In [ ]:
def generate_access_token():
    headers={}
    headers["Accept"] = "application/json"
    auth = HTTPBasicAuth(WOS_CREDENTIALS["username"], WOS_CREDENTIALS["password"])
    
    ICP_TOKEN_URL= WOS_CREDENTIALS["url"] + "/v1/preauth/validateAuth"
    
    response = requests.get(ICP_TOKEN_URL, headers=headers, auth=auth, verify=False)
    json_data = response.json()
    icp_access_token = json_data['accessToken']
    return icp_access_token

## Bind WML machine learning instance as Pre-Prod

Watson OpenScale needs to be bound to the Watson Machine Learning instance to capture payload data into and out of the model. If a binding with name "WML Pre-Prod" already exists, this code will delete that binding a create a new one.

In [ ]:
all_bindings = ai_client.data_mart.bindings.get_details()['service_bindings']
for binding in all_bindings:
    if binding['entity']['name'] == "WML Pre-Prod":
        binding_uid = binding['metadata']['guid']
        ai_client.data_mart.bindings.delete(binding_uid)

In [ ]:
import uuid

headers = {}
headers["Content-Type"] = "application/json"
headers["Authorization"] = "Bearer {}".format(generate_access_token())

payload = {
    "name": "WML Pre-Prod",
    "description": "WML Instance designated as Pre-Production",
    "service_type": "watson_machine_learning",
    "instance_id": str(uuid.uuid4()),
    "credentials": {
    },
    "operational_space_id": "pre_production",
    "deployment_space_id": space_id
}

binding_uid = str(uuid.uuid4())

SERVICE_BINDINGS_URL = WOS_CREDENTIALS["url"] + "/v1/data_marts/{0}/service_bindings/{1}".format(WOS_GUID, binding_uid)

response = requests.put(SERVICE_BINDINGS_URL, json=payload, headers=headers, verify=False)
json_data = response.json()

In [ ]:
print(binding_uid)

In [ ]:
ai_client.data_mart.bindings.get_details(binding_uid)

## Create an integration to IBM OpenPages

To push metrics from Watson OpenScale to IBM OpenPages, a connection must be established between the two services.

In [ ]:
headers = {}
headers["Content-Type"] = "application/json"
headers["Authorization"] = "Bearer {}".format(generate_access_token())

INTEGRATED_SYSTEMS_URL = WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/integrated_systems".format(WOS_GUID)

payload = {
    "name": "OpenPages Connection",
    "type": "open_pages",
    "description": "Integration with OpenPages",
    "credentials": OPENPAGES_CREDENTIALS
}

response = requests.post(INTEGRATED_SYSTEMS_URL, json=payload, headers=headers, verify=False)
json_data = response.json()
print(json_data)

if "metadata" in json_data and "id" in json_data["metadata"]:
    integrated_system_id = json_data["metadata"]["id"]
    print(integrated_system_id)

## Subscriptions
### Remove existing PreProd and Challenger credit risk subscriptions
This code removes previous subscriptions to the German Credit model to refresh the monitors with the new model and new data.

In [ ]:
subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
for subscription in subscriptions_uids:
    sub_name = ai_client.data_mart.subscriptions.get_details(subscription)['entity']['asset']['name']
    if sub_name == PRE_PROD_MODEL_NAME or sub_name == PRE_PROD_CHALLENGER_MODEL_NAME:
        ai_client.data_mart.subscriptions.delete(subscription)
        print('Deleted existing subscription for', sub_name)

In [ ]:
pre_prod_subscription = ai_client.data_mart.subscriptions.add(WatsonMachineLearningAsset(
    pre_prod_model_uid,
    binding_uid=binding_uid,
    problem_type=ProblemType.BINARY_CLASSIFICATION,
    input_data_type=InputDataType.STRUCTURED,
    label_column='Risk',
    prediction_column='predictedLabel',
    probability_column='probability',
    feature_columns = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"],
    categorical_columns = ["CheckingStatus","CreditHistory","LoanPurpose","ExistingSavings","EmploymentDuration","Sex","OthersOnLoan","OwnsProperty","InstallmentPlans","Housing","Job","Telephone","ForeignWorker"]
))

if pre_prod_subscription is None:
    print('Subscription already exists; get the existing one')
    subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
    for sub in subscriptions_uids:
        if ai_client.data_mart.subscriptions.get_details(sub)['entity']['asset']['name'] == PRE_PROD_MODEL_NAME:
            pre_prod_subscription = ai_client.data_mart.subscriptions.get(sub)

In [ ]:
challenger_subscription = ai_client.data_mart.subscriptions.add(WatsonMachineLearningAsset(
    challenger_model_uid,
    binding_uid=binding_uid,
    problem_type=ProblemType.BINARY_CLASSIFICATION,
    input_data_type=InputDataType.STRUCTURED,
    label_column='Risk',
    prediction_column='prediction',
    probability_column='probability',
    feature_columns = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"],
    categorical_columns = ["CheckingStatus","CreditHistory","LoanPurpose","ExistingSavings","EmploymentDuration","Sex","OthersOnLoan","OwnsProperty","InstallmentPlans","Housing","Job","Telephone","ForeignWorker"]
))

if challenger_subscription is None:
    print('Subscription already exists; get the existing one')
    subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
    for sub in subscriptions_uids:
        if ai_client.data_mart.subscriptions.get_details(sub)['entity']['asset']['name'] == PRE_PROD_CHALLENGER_MODEL_NAME:
            challenger_subscription = ai_client.data_mart.subscriptions.get(sub)

In [ ]:
ai_client.data_mart.subscriptions.list()

In [ ]:
pre_prod_subscription.uid

In [ ]:
challenger_subscription.uid

## Patch the training data reference in both the preprod & challenger subscription

In [ ]:
headers = {}
headers["Content-Type"] = "application/json"
headers["Authorization"] = "Bearer {}".format(generate_access_token())

training_data_reference = {
  "connection": {
    "connection_string": "jdbc:db2://dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net:50000/BLUDB:retrieveMessagesFromServerOnGetMessage=true;",
    "database_name": "BLUDB",
    "hostname": "dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net",
    "password": "khhz72v+6mcwwkfv",
    "username": "cmb91569"
  },
  "location": {
    "schema_name": "CMB91569",
    "table_name": "CREDIT_RISK_TRAIN_DATA"
  },
  "name": "German credit risk training data",
  "type": "db2"
}

payload = [
 {
   "op": "replace",
   "path": "/asset_properties/training_data_reference",
   "value": training_data_reference
 }
]

In [ ]:
SUBSCRIPTION_URL = WOS_CREDENTIALS["url"] + "/v1/data_marts/{0}/service_bindings/{1}/subscriptions/{2}".format(WOS_GUID, binding_uid, pre_prod_subscription.uid)

response = requests.patch(SUBSCRIPTION_URL, json=payload, headers=headers, verify=False)
json_data = response.json()
print(json_data)

In [ ]:
SUBSCRIPTION_URL = WOS_CREDENTIALS["url"] + "/v1/data_marts/{0}/service_bindings/{1}/subscriptions/{2}".format(WOS_GUID, binding_uid, challenger_subscription.uid)

response = requests.patch(SUBSCRIPTION_URL, json=payload, headers=headers, verify=False)
json_data = response.json()
print(json_data)

### Score the model so we can configure monitors
Now that the WML service has been bound and the subscription has been created, we need to send a request to the model before we configure OpenScale. This allows OpenScale to create a payload log in the datamart with the correct schema, so it can capture data coming into and out of the model. First, the code gets the model deployment's endpoint URL, and then sends a few records for predictions.

In [ ]:
fields = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"]
values = [
  ["no_checking",13,"credits_paid_to_date","car_new",1343,"100_to_500","1_to_4",2,"female","none",3,"savings_insurance",46,"none","own",2,"skilled",1,"none","yes"],
  ["no_checking",24,"prior_payments_delayed","furniture",4567,"500_to_1000","1_to_4",4,"male","none",4,"savings_insurance",36,"none","free",2,"management_self-employed",1,"none","yes"],
  ["0_to_200",26,"all_credits_paid_back","car_new",863,"less_100","less_1",2,"female","co-applicant",2,"real_estate",38,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",14,"no_credits","car_new",2368,"less_100","1_to_4",3,"female","none",3,"real_estate",29,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",4,"no_credits","car_new",250,"less_100","unemployed",2,"female","none",3,"real_estate",23,"none","rent",1,"management_self-employed",1,"none","yes"],
  ["no_checking",17,"credits_paid_to_date","car_new",832,"100_to_500","1_to_4",2,"male","none",2,"real_estate",42,"none","own",1,"skilled",1,"none","yes"],
  ["no_checking",33,"outstanding_credit","appliances",5696,"unknown","greater_7",4,"male","co-applicant",4,"unknown",54,"none","free",2,"skilled",1,"yes","yes"],
  ["0_to_200",13,"prior_payments_delayed","retraining",1375,"100_to_500","4_to_7",3,"male","none",3,"real_estate",37,"none","own",2,"management_self-employed",1,"none","yes"]
]

payload_scoring = {"fields": fields,"values": values}

In [ ]:
payload = {
    wml_client.deployments.ScoringMetaNames.INPUT_DATA: [payload_scoring]
}
scoring_response = wml_client.deployments.score(pre_prod_deployment_uid, payload)

print('Single record scoring result:', '\n fields:', scoring_response['predictions'][0]['fields'], '\n values: ', scoring_response['predictions'][0]['values'][0])

In [ ]:
time.sleep(10)
pre_prod_subscription.payload_logging.get_records_count()

In [ ]:
payload = {
    wml_client.deployments.ScoringMetaNames.INPUT_DATA: [payload_scoring]
}
scoring_response = wml_client.deployments.score(challenger_deployment_uid, payload)

print('Single record scoring result:', '\n fields:', scoring_response['predictions'][0]['fields'], '\n values: ', scoring_response['predictions'][0]['values'][0])

In [ ]:
time.sleep(10)
challenger_subscription.payload_logging.get_records_count()

# Quality monitoring

## Enable quality monitoring
The code below waits ten seconds to allow the payload logging table to be set up before it begins enabling monitors. First, it turns on the quality (accuracy) monitor and sets an alert threshold of 80%. OpenScale will show an alert on the dashboard if the model accuracy measurement (area under the curve, in the case of a binary classifier) falls below this threshold.

The second paramater supplied, min_records, specifies the minimum number of feedback records OpenScale needs before it calculates a new measurement. The quality monitor runs hourly, but the accuracy reading in the dashboard will not change until an additional 50 feedback records have been added, via the user interface, the Python client, or the supplied feedback endpoint.

In [ ]:
time.sleep(10)
pre_prod_subscription.quality_monitoring.enable(threshold=0.8, min_records=100)

In [ ]:
time.sleep(10)
challenger_subscription.quality_monitoring.enable(threshold=0.8, min_records=100)

# Fairness, drift monitoring and explanations 

## Fairness configuration
The code below configures fairness monitoring for our model. It turns on monitoring for two features, Sex and Age. In each case, we must specify:

Which model feature to monitor
One or more majority groups, which are values of that feature that we expect to receive a higher percentage of favorable outcomes
One or more minority groups, which are values of that feature that we expect to receive a higher percentage of unfavorable outcomes
The threshold at which we would like OpenScale to display an alert if the fairness measurement falls below (in this case, 80%)
Additionally, we must specify which outcomes from the model are favourable outcomes, and which are unfavourable. We must also provide the number of records OpenScale will use to calculate the fairness score. In this case, OpenScale's fairness monitor will run hourly, but will not calculate a new fairness rating until at least 100 records have been added. Finally, to calculate fairness, OpenScale must perform some calculations on the training data, so we provide the dataframe containing the data.

In [ ]:
import pandas as pd
pd_data = pd.read_csv("german_credit_data_biased_training.csv", sep=",", header=0)

pre_prod_subscription.fairness_monitoring.enable(
            features=[
                Feature("Sex", majority=['male'], minority=['female'], threshold=0.80),
                Feature("Age", majority=[[26,74]], minority=[[19,25]], threshold=0.80)
            ],
            favourable_classes=['No Risk'],
            unfavourable_classes=['Risk'],
            min_records=100,
            training_data=pd_data
        )

In [ ]:
challenger_subscription.fairness_monitoring.enable(
            features=[
                Feature("Sex", majority=['male'], minority=['female'], threshold=0.80),
                Feature("Age", majority=[[26,74]], minority=[[19,25]], threshold=0.80)
            ],
            favourable_classes=['No Risk'],
            unfavourable_classes=['Risk'],
            min_records=100,
            training_data=pd_data
        )

## Drift configuration

Enable the drift configuration for both the subscription created with a threshold of 10% and minimal sample as 100 records.

In [ ]:
pre_prod_subscription.drift_monitoring.enable(threshold=0.10, min_records=100)

In [ ]:
drift_status = None
while drift_status != 'finished':
    drift_details = pre_prod_subscription.drift_monitoring.get_details()
    drift_status = drift_details['parameters']['config_status']['state']
    if drift_status != 'finished':
        print(datetime.utcnow().strftime('%H:%M:%S'), drift_status)
        time.sleep(30)
print(drift_status)

In [ ]:
challenger_subscription.drift_monitoring.enable(threshold=0.10, min_records=100)

In [ ]:
drift_status = None
while drift_status != 'finished':
    drift_details = challenger_subscription.drift_monitoring.get_details()
    drift_status = drift_details['parameters']['config_status']['state']
    if drift_status != 'finished':
        print(datetime.utcnow().strftime('%H:%M:%S'), drift_status)
        time.sleep(30)
print(drift_status)

## Configure Explainability
Finally, we provide OpenScale with the training data to enable and configure the explainability features.

In [ ]:
from ibm_ai_openscale.supporting_classes import *

pre_prod_subscription.explainability.enable(training_data=pd_data)
challenger_subscription.explainability.enable(training_data=pd_data)

## Enable MRM 

We enable the MRM configuration for both the subscriptions

In [ ]:
headers = {}
headers["Content-Type"] = "application/json"
headers["Authorization"] = "Bearer {}".format(generate_access_token())

payload = {
  "data_mart_id": WOS_GUID,
  "monitor_definition_id": "mrm",
  "target": {
    "target_id": pre_prod_subscription.uid,
    "target_type": "subscription"
  },
  "parameters": {
  },
  "managed_by": "user"
}

MONITOR_INSTANCES_URL = WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/monitor_instances".format(WOS_GUID)

response = requests.post(MONITOR_INSTANCES_URL, json=payload, headers=headers, verify=False)
json_data = response.json()
print(json_data)
if "metadata" in json_data and "id" in json_data["metadata"]:
    pre_prod_mrm_instance_id = json_data["metadata"]["id"]

In [ ]:
headers = {}
headers["Content-Type"] = "application/json"
headers["Authorization"] = "Bearer {}".format(generate_access_token())

payload = {
  "data_mart_id": WOS_GUID,
  "monitor_definition_id": "mrm",
  "target": {
    "target_id": challenger_subscription.uid,
    "target_type": "subscription"
  },
  "parameters": {
  },
  "managed_by": "user"
}

MONITOR_INSTANCES_URL = WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/monitor_instances".format(WOS_GUID)

response = requests.post(MONITOR_INSTANCES_URL, json=payload, headers=headers, verify=False)
json_data = response.json()
print(json_data)
if "metadata" in json_data and "id" in json_data["metadata"]:
    challenger_mrm_instance_id = json_data["metadata"]["id"]

## Patch the integration reference in the pre-prod subscription

In [ ]:
headers = {}
headers["Content-Type"] = "application/json"
headers["Authorization"] = "Bearer {}".format(generate_access_token())

payload = [
  {
    "op": "add",
    "path": "/integration_reference",
    "value": {
        "integrated_system_id": integrated_system_id,
        "external_id": openpages_model_id
    }
  }
]

SUBSCRIPTIONS_URL = WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/subscriptions/{1}".format(WOS_GUID, pre_prod_subscription.uid)

response = requests.patch(SUBSCRIPTIONS_URL, json=payload, headers=headers, verify=False)
json_data = response.json()

## Create test data sets from the training data 

In [ ]:
test_data_1 = pd_data[1:201]
test_data_1.to_csv("german_credit_risk_test_data_1.csv", encoding="utf-8", index=False)
test_data_2 = pd_data[201:401]
test_data_2.to_csv("german_credit_risk_test_data_2.csv", encoding="utf-8", index=False)
test_data_3 = pd_data[401:601]
test_data_3.to_csv("german_credit_risk_test_data_3.csv", encoding="utf-8", index=False)
test_data_4 = pd_data[601:801]
test_data_4.to_csv("german_credit_risk_test_data_4.csv", encoding="utf-8", index=False)

## Function to upload, evaluate and check the status of the evaluation

This function will upload the test data CSV and trigger the risk evaluation. It will iterate and check the status of the evaluation until its finished with a finite wait duration

In [ ]:
def upload_and_evaluate(file_name, mrm_instance_id):
    
    print("Running upload and evaluate for {}".format(file_name))
    import json
    import time
    from datetime import datetime

    status = None
    monitoring_run_id = None
    GET_UPLOAD_AND_EVALUATION_STATUS_RETRIES = 32
    GET_UPLOAD_AND_EVALUATION_STATUS_INTERVAL = 10
    
    if file_name is not None:
        
        headers = {}
        headers["Content-Type"] = "text/csv"
        headers["Authorization"] = "Bearer {}".format(generate_access_token())
        
        POST_EVALUATIONS_URL = WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/monitoring_services/mrm/monitor_instances/{1}/risk_evaluations?test_data_set_name={2}".format(WOS_GUID, mrm_instance_id, file_name)

        with open(file_name) as file:
            f = file.read()
            b = bytearray(f, 'utf-8')

        response = requests.post(POST_EVALUATIONS_URL, data=bytes(b), headers=headers, verify=False)
        if response.ok is False:
            print("Upload and evalaute for {0} failed with {1}: {2}".format(file_name, response.status_code, response.reason))
            return
        
        headers = {}
        headers["Content-Type"] = "application/json"
        headers["Authorization"] = "Bearer {}".format(generate_access_token())

        GET_EVALUATIONS_URL = WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/monitoring_services/mrm/monitor_instances/{1}/risk_evaluations".format(WOS_GUID, mrm_instance_id)
        
        for i in range(GET_UPLOAD_AND_EVALUATION_STATUS_RETRIES):
        
            response = requests.get(GET_EVALUATIONS_URL, headers=headers, verify=False)
            if response.ok is False:
                print("Getting status of upload and evalaute for {0} failed with {1}: {2}".format(file_name, response.status_code, response.reason))
                return

            response = json.loads(response.text)
            if "metadata" in response and "id" in response["metadata"]:
                monitoring_run_id = response["metadata"]["id"]
            if "entity" in response and "status" in response["entity"]:
                status = response["entity"]["status"]["state"]
            
            if status is not None:
                print(datetime.utcnow().strftime('%H:%M:%S'), status.lower())
                if status.lower() in ["finished", "completed"]:
                    break
                elif "error" in status.lower():
                    print(response)
                    break

            time.sleep(GET_UPLOAD_AND_EVALUATION_STATUS_INTERVAL)

    return status, monitoring_run_id

## Perform Risk Evaluations

We now start performing evaluations of smaller data sets against both the PreProd and Challenger subscriptions

In [ ]:
upload_and_evaluate("german_credit_risk_test_data_1.csv", pre_prod_mrm_instance_id)

In [ ]:
upload_and_evaluate("german_credit_risk_test_data_2.csv", pre_prod_mrm_instance_id)

In [ ]:
upload_and_evaluate("german_credit_risk_test_data_3.csv", pre_prod_mrm_instance_id)

In [ ]:
upload_and_evaluate("german_credit_risk_test_data_4.csv", pre_prod_mrm_instance_id)

In [ ]:
upload_and_evaluate("german_credit_risk_test_data_1.csv", challenger_mrm_instance_id)

In [ ]:
upload_and_evaluate("german_credit_risk_test_data_2.csv", challenger_mrm_instance_id)

In [ ]:
upload_and_evaluate("german_credit_risk_test_data_3.csv", challenger_mrm_instance_id)

In [ ]:
upload_and_evaluate("german_credit_risk_test_data_4.csv", challenger_mrm_instance_id)

## Explore the Model Risk Management UI

Here is a quick recap of what we have done so far.

1. We've deployed two Credit Risk Model to a WML instance that is designated as Pre-Production
2. We've created subscriptions of these two model deployments in OpenScale
3. Configured all monitors supported by OpenScale for these subscriptions
4. We've performed a few risk evaluations against both these susbscription with the same set of test data

Now, pleaase explore the MRM UI to visualize the results, compare the performance of models, download the evaluation report as PDF

Link to OpenScale : https://CLUSTER_URL/aiopenscale/insights

# Promote pre-production model to production 

After you have reviewed the evaluation results of the PreProd Vs Challenger and if you make the decision to promote the Challenger model to Production, the first thing you need to do is to deploy the model into a WML instance that is designated as Production instance

## Deploy model to production WML instance 

In [ ]:
PROD_MODEL_NAME="German Credit Risk Model - Prod"
PROD_DEPLOYMENT_NAME="German Credit Risk Model - Prod"

PROD_SPACE_NAME="prod"

In [ ]:
wml_client.spaces.list()

# Find and set the default space
space_name=PROD_SPACE_NAME
spaces = wml_client.spaces.get_details()['resources']
space_id = None
for space in spaces:
    if space['entity']['name'] == space_name:
        space_id = space["metadata"]["guid"]
if space_id is None:
    space_id = wml_client.spaces.store(
        meta_props={wml_client.spaces.ConfigurationMetaNames.NAME: space_name})["metadata"]["guid"]
wml_client.set.default_space(space_id)

In [ ]:
import numpy 
numpy.version.version

import pandas as pd
import json

from pyspark import SparkContext, SQLContext
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier,GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import StringIndexer, VectorAssembler, IndexToString
from pyspark.sql.types import StructType, DoubleType, StringType, ArrayType

from pyspark.sql import SparkSession
from pyspark import SparkFiles

spark = SparkSession.builder.getOrCreate()
pd_data = pd.read_csv("german_credit_data_biased_training.csv", sep=",", header=0)
spark_df = spark.read.csv(path="german_credit_data_biased_training.csv", sep=",", header=True, inferSchema=True)
spark_df.head()

(train_data, test_data) = spark_df.randomSplit([0.9, 0.1], 24)
print("Number of records for training: " + str(train_data.count()))
print("Number of records for evaluation: " + str(test_data.count()))

si_CheckingStatus = StringIndexer(inputCol='CheckingStatus', outputCol='CheckingStatus_IX')
si_CreditHistory = StringIndexer(inputCol='CreditHistory', outputCol='CreditHistory_IX')
si_LoanPurpose = StringIndexer(inputCol='LoanPurpose', outputCol='LoanPurpose_IX')
si_ExistingSavings = StringIndexer(inputCol='ExistingSavings', outputCol='ExistingSavings_IX')
si_EmploymentDuration = StringIndexer(inputCol='EmploymentDuration', outputCol='EmploymentDuration_IX')
si_Sex = StringIndexer(inputCol='Sex', outputCol='Sex_IX')
si_OthersOnLoan = StringIndexer(inputCol='OthersOnLoan', outputCol='OthersOnLoan_IX')
si_OwnsProperty = StringIndexer(inputCol='OwnsProperty', outputCol='OwnsProperty_IX')
si_InstallmentPlans = StringIndexer(inputCol='InstallmentPlans', outputCol='InstallmentPlans_IX')
si_Housing = StringIndexer(inputCol='Housing', outputCol='Housing_IX')
si_Job = StringIndexer(inputCol='Job', outputCol='Job_IX')
si_Telephone = StringIndexer(inputCol='Telephone', outputCol='Telephone_IX')
si_ForeignWorker = StringIndexer(inputCol='ForeignWorker', outputCol='ForeignWorker_IX')
si_Label = StringIndexer(inputCol="Risk", outputCol="label").fit(spark_df)
label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=si_Label.labels)

va_features = VectorAssembler(
inputCols=["CheckingStatus_IX", "CreditHistory_IX", "LoanPurpose_IX", "ExistingSavings_IX",
           "EmploymentDuration_IX", "Sex_IX", "OthersOnLoan_IX", "OwnsProperty_IX", "InstallmentPlans_IX",
           "Housing_IX", "Job_IX", "Telephone_IX", "ForeignWorker_IX", "LoanDuration", "LoanAmount",
           "InstallmentPercent", "CurrentResidenceDuration", "LoanDuration", "Age", "ExistingCreditsCount",
           "Dependents"], outputCol="features")

classifier=GBTClassifier(featuresCol="features")

pipeline = Pipeline(
stages=[si_CheckingStatus, si_CreditHistory, si_EmploymentDuration, si_ExistingSavings, si_ForeignWorker,
        si_Housing, si_InstallmentPlans, si_Job, si_LoanPurpose, si_OthersOnLoan,
        si_OwnsProperty, si_Sex, si_Telephone, si_Label, va_features, classifier, label_converter])

model = pipeline.fit(train_data)
predictions = model.transform(test_data)
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction")
auc = evaluator.evaluate(predictions)

print("Accuracy = %g" % auc)

# Remove existing model and deployment
MODEL_NAME=PROD_MODEL_NAME
DEPLOYMENT_NAME=PROD_DEPLOYMENT_NAME

deployment_details = wml_client.deployments.get_details()
for deployment in deployment_details['resources']:
    deployment_id = deployment['metadata']['guid']
    model_id = deployment['entity']['asset']['href'].split('/')[3].split('?')[0]
    if deployment['entity']['name'] == DEPLOYMENT_NAME:
        print('Deleting deployment id', deployment_id)
        wml_client.deployments.delete(deployment_id)
        print('Deleting model id', model_id)
        wml_client.repository.delete(model_id)
wml_client.repository.list_models()

# Save Model
model_props_rf = {
    wml_client.repository.ModelMetaNames.NAME: MODEL_NAME,
    wml_client.repository.ModelMetaNames.DESCRIPTION: MODEL_NAME,
    wml_client.repository.ModelMetaNames.RUNTIME_UID: "spark-mllib_2.3",
    wml_client.repository.ModelMetaNames.TYPE: "mllib_2.3"
}

published_model_details = wml_client.repository.store_model(model=model, meta_props=model_props_rf, training_data=train_data, pipeline=pipeline)
print(published_model_details)

# List models in the repository
wml_client.repository.list_models()

# Get the model UID
prod_model_uid = wml_client.repository.get_model_uid(published_model_details)
prod_model_uid


# Deploy model
wml_deployments = wml_client.deployments.get_details()
prod_deployment_uid = None
for deployment in wml_deployments['resources']:
    if DEPLOYMENT_NAME == deployment['entity']['name']:
        prod_deployment_uid = deployment['metadata']['guid']
        break

if prod_deployment_uid is None:
    print("Deploying model...")
    meta_props = {
        wml_client.deployments.ConfigurationMetaNames.NAME: DEPLOYMENT_NAME,
        wml_client.deployments.ConfigurationMetaNames.DESCRIPTION: DEPLOYMENT_NAME,
        wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
    deployment = wml_client.deployments.create(artifact_uid=prod_model_uid, name=DEPLOYMENT_NAME, meta_props=meta_props)
    prod_deployment_uid = wml_client.deployments.get_uid(deployment)

print("Model id: {}".format(prod_model_uid))
print("Deployment id: {}".format(prod_deployment_uid))

prod_deployment_uid=wml_client.deployments.get_uid(deployment)
prod_deployment_uid

fields = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"]
values = [
  ["no_checking",13,"credits_paid_to_date","car_new",1343,"100_to_500","1_to_4",2,"female","none",3,"savings_insurance",46,"none","own",2,"skilled",1,"none","yes"],
  ["no_checking",24,"prior_payments_delayed","furniture",4567,"500_to_1000","1_to_4",4,"male","none",4,"savings_insurance",36,"none","free",2,"management_self-employed",1,"none","yes"],
  ["0_to_200",26,"all_credits_paid_back","car_new",863,"less_100","less_1",2,"female","co-applicant",2,"real_estate",38,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",14,"no_credits","car_new",2368,"less_100","1_to_4",3,"female","none",3,"real_estate",29,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",4,"no_credits","car_new",250,"less_100","unemployed",2,"female","none",3,"real_estate",23,"none","rent",1,"management_self-employed",1,"none","yes"],
  ["no_checking",17,"credits_paid_to_date","car_new",832,"100_to_500","1_to_4",2,"male","none",2,"real_estate",42,"none","own",1,"skilled",1,"none","yes"],
  ["no_checking",33,"outstanding_credit","appliances",5696,"unknown","greater_7",4,"male","co-applicant",4,"unknown",54,"none","free",2,"skilled",1,"yes","yes"],
  ["0_to_200",13,"prior_payments_delayed","retraining",1375,"100_to_500","4_to_7",3,"male","none",3,"real_estate",37,"none","own",2,"management_self-employed",1,"none","yes"]
]

payload_scoring = {"fields": fields,"values": values}
payload = {
    wml_client.deployments.ScoringMetaNames.INPUT_DATA: [payload_scoring]
}
scoring_response = wml_client.deployments.score(prod_deployment_uid, payload)

print('Single record scoring result:', '\n fields:', scoring_response['predictions'][0]['fields'], '\n values: ', scoring_response['predictions'][0]['values'][0])

## Bind WML machine learning instance as Prod

Watson OpenScale needs to be bound to the Watson Machine Learning instance to capture payload data into and out of the model. If a binding with name "WML Prod" already exists, this code will delete that binding a create a new one.

In [ ]:
all_bindings = ai_client.data_mart.bindings.get_details()['service_bindings']
for binding in all_bindings:
    if binding['entity']['name'] == "WML Prod":
        binding_uid = binding['metadata']['guid']
        ai_client.data_mart.bindings.delete(binding_uid)

In [ ]:
import uuid

headers = {}
headers["Content-Type"] = "application/json"
headers["Authorization"] = "Bearer {}".format(generate_access_token())

payload = {
    "name": "WML Prod",
    "description": "WML Instance designated as Production",
    "service_type": "watson_machine_learning",
    "instance_id": str(uuid.uuid4()),
    "credentials": {
    },
    "operational_space_id": "production",
    "deployment_space_id": space_id
}

binding_uid = str(uuid.uuid4())

SERVICE_BINDINGS_URL = WOS_CREDENTIALS["url"] + "/v1/data_marts/{0}/service_bindings/{1}".format(WOS_GUID, binding_uid)

response = requests.put(SERVICE_BINDINGS_URL, json=payload, headers=headers, verify=False)
json_data = response.json()

In [ ]:
print(binding_uid)
ai_client.data_mart.bindings.get_details(binding_uid)

# Import configuration settings from pre-prod model

With MRM we provide a important feature that lets you copy the configuration settings of your pre-production subscription to the production subscription. To try this out

1. Navigate to Model Monitors view in Insights dashboard of OpenScale
2. Click on the Add to dashboard
3. Select the production model deployment from WML production machine learning provider and click on Configure
4. In Selections saved dialog, click on Configure monitors
4. In Model Governance, click on Begin and choose the same OpenPages model that was linked to the pre-production model and click on Save
5. You will be presented with an dialog about the OpenPages model being associated with another subscription and if you'd like to import settings.
6. Click on Import and confirm

All the configuration settings are now copied into the production subscription


<b>Note: The next set of cells should be executed only after finishing the import settings from the OpenScale dashboard</b>

## Score the production model so that we can trigger monitors

Now that the production subscription is configured by copying the configuration, there would be schedules created for each of the monitors to run on a scheduled basis. 
Quality, Fairness and Mrm will run hourly. Drift will run once in three hours.

For this demo purpose, we will trigger the monitors on-demand so that we can see the model summary dashboard without having to wait the entire hour. 
To do that lets first push some records in the Payload Logging table.

In [ ]:
df = pd_data.sample(n=400)
df = df.drop(['Risk'], axis=1)
fields = df.columns.tolist()
values = df.values.tolist()

payload_scoring = {"fields": fields,"values": values}

In [ ]:
payload = {
    wml_client.deployments.ScoringMetaNames.INPUT_DATA: [payload_scoring]
}
scoring_response = wml_client.deployments.score(prod_deployment_uid, payload)

print('Single record scoring result:', '\n fields:', scoring_response['predictions'][0]['fields'], '\n values: ', scoring_response['predictions'][0]['values'][0])

In [ ]:
prod_subscription = ai_client.data_mart.subscriptions.get(name=PROD_DEPLOYMENT_NAME)
prod_subscription.uid

In [ ]:
time.sleep(10)
prod_subscription.payload_logging.get_records_count()

## Fetch all monitor instances

In [ ]:
headers = {}
headers["Content-Type"] = "application/json"
headers["Authorization"] = "Bearer {}".format(generate_access_token())

MONITOR_INSTANCES_URL = WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/monitor_instances?target.target_id={1}&target.target_type=subscription".format(WOS_GUID, prod_subscription.uid)
print(MONITOR_INSTANCES_URL)

response = requests.get(MONITOR_INSTANCES_URL, headers=headers, verify=False)
monitor_instances = response.json()["monitor_instances"]

drift_monitor_instance_id = None
quality_monitor_instance_id = None
mrm_monitor_instance_id = None

if monitor_instances is not None:
    for monitor_instance in monitor_instances:
        if "entity" in monitor_instance and "monitor_definition_id" in monitor_instance["entity"]:
            monitor_name = monitor_instance["entity"]["monitor_definition_id"]
            if "metadata" in monitor_instance and "id" in monitor_instance["metadata"]:
                id = monitor_instance["metadata"]["id"]
                if monitor_name == "drift":
                    drift_monitor_instance_id = id
                elif monitor_name == "quality":
                    quality_monitor_instance_id = id
                elif monitor_name == "mrm":
                    mrm_monitor_instance_id = id
                    
print("Quality monitor instance id - {0}".format(quality_monitor_instance_id))
print("Drift monitor instance id - {0}".format(drift_monitor_instance_id))
print("MRM monitor instance id - {0}".format(mrm_monitor_instance_id))

## Function to get the monitoring run details

In [ ]:
def get_monitoring_run_details(monitor_instance_id, monitoring_run_id):
    
    headers = {}
    headers["Content-Type"] = "application/json"
    headers["Authorization"] = "Bearer {}".format(generate_access_token())
    
    MONITORING_RUNS_URL = WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/monitor_instances/{1}/runs/{2}".format(WOS_GUID, monitor_instance_id, monitoring_run_id)
    response = requests.get(MONITORING_RUNS_URL, headers=headers, verify=False)
    return response.json()

## Run on-demand Quality

In [ ]:
headers = {}
headers["Content-Type"] = "application/json"
headers["Authorization"] = "Bearer {}".format(generate_access_token())

if quality_monitor_instance_id is not None:
    MONITOR_RUN_URL = WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/monitor_instances/{1}/runs".format(WOS_GUID, quality_monitor_instance_id)
    payload = {
        "triggered_by": "user"
    }
    print("Triggering Quality computation with {}".format(MONITOR_RUN_URL))
    response = requests.post(MONITOR_RUN_URL, json=payload, headers=headers, verify=False)
    json_data = response.json()
    if "metadata" in json_data and "id" in json_data["metadata"]:
        quality_monitoring_run_id = json_data["metadata"]["id"]
    print("Done triggering Quality computation")

In [ ]:
from datetime import datetime

quality_run_status = None
while quality_run_status != 'finished':
    monitoring_run_details = get_monitoring_run_details(quality_monitor_instance_id, quality_monitoring_run_id)
    quality_run_status = monitoring_run_details["entity"]["status"]["state"]
    if quality_run_status == "error":
        print(monitoring_run_details)
        break
    if quality_run_status != 'finished':
        print(datetime.utcnow().strftime('%H:%M:%S'), quality_run_status)
        time.sleep(10)
print(quality_run_status)

## Run on-demand Drift

In [ ]:
headers = {}
headers["Content-Type"] = "application/json"
headers["Authorization"] = "Bearer {}".format(generate_access_token())

if drift_monitor_instance_id is not None:
    MONITOR_RUN_URL = WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/monitor_instances/{1}/runs".format(WOS_GUID, drift_monitor_instance_id)
    payload = {
        "triggered_by": "user"
    }
    print("Triggering Drift computation with {}".format(MONITOR_RUN_URL))
    response = requests.post(MONITOR_RUN_URL, json=payload, headers=headers, verify=False)
    json_data = response.json()
    if "metadata" in json_data and "id" in json_data["metadata"]:
        drift_monitoring_run_id = json_data["metadata"]["id"]
    print("Done triggering Drift computation")

In [ ]:
from datetime import datetime

drift_run_status = None
while drift_run_status != 'finished':
    monitoring_run_details = get_monitoring_run_details(drift_monitor_instance_id, drift_monitoring_run_id)
    drift_run_status = monitoring_run_details["entity"]["status"]["state"]
    if drift_run_status == "error":
        print(monitoring_run_details)
        break
    if drift_run_status != 'finished':
        print(datetime.utcnow().strftime('%H:%M:%S'), drift_run_status)
        time.sleep(10)
print(drift_run_status)

## Run on-demand Fairness

In [ ]:
headers = {}
headers["Content-Type"] = "application/json"
headers["Authorization"] = "Bearer {}".format(generate_access_token())

FAIRNESS_RUNS_URL = WOS_CREDENTIALS["url"] + "/v1/fairness_monitoring/{0}/runs".format(prod_subscription.uid)
payload = {
  "binding_id": binding_uid,
  "subscription_id": prod_subscription.uid,
  "deployment_id": prod_deployment_uid,
  "data_mart_id": WOS_GUID
}

print("Triggering Fairness computation with {}".format(FAIRNESS_RUNS_URL))
response = requests.post(FAIRNESS_RUNS_URL, json=payload, headers=headers, verify=False)
jsonData = response.json()
print("Done triggering Fairness computation")

In [ ]:
from datetime import datetime

fairness_run_status = None
while fairness_run_status != 'FINISHED':
    fairness_monitoring_details = prod_subscription.fairness_monitoring.get_details()
    fairness_run_status = fairness_monitoring_details["parameters"]["run_status"][prod_deployment_uid]["run_status"]
    if fairness_run_status == "FINISHED WITH ERRORS":
        print(fairness_monitoring_details)
        break
    if fairness_run_status != 'FINISHED':
        print(datetime.utcnow().strftime('%H:%M:%S'), fairness_run_status)
        time.sleep(10)
print(fairness_run_status)

## Run on-demand MRM

In [ ]:
headers = {}
headers["Content-Type"] = "application/json"
headers["Authorization"] = "Bearer {}".format(generate_access_token())

if mrm_monitor_instance_id is not None:
    MONITOR_RUN_URL = WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/monitor_instances/{1}/runs".format(WOS_GUID, mrm_monitor_instance_id)
    payload = {
        "triggered_by": "user"
    }
    print("Triggering MRM computation with {}".format(MONITOR_RUN_URL))
    response = requests.post(MONITOR_RUN_URL, json=payload, headers=headers, verify=False)
    json_data = response.json()
    if "metadata" in json_data and "id" in json_data["metadata"]:
        mrm_monitoring_run_id = json_data["metadata"]["id"]
    print("Done triggering MRM computation")

In [ ]:
from datetime import datetime

mrm_run_status = None
while mrm_run_status != 'finished':
    monitoring_run_details = get_monitoring_run_details(mrm_monitor_instance_id, mrm_monitoring_run_id)
    mrm_run_status = monitoring_run_details["entity"]["status"]["state"]
    if mrm_run_status == "error":
        print(monitoring_run_details)
        break
    if mrm_run_status != 'finished':
        print(datetime.utcnow().strftime('%H:%M:%S'), mrm_run_status)
        time.sleep(10)
print(mrm_run_status)

## Refresh the model summary of the production subscription in the OpenScale dashboard

This brings us to the end of this demo exercise. Thank you for trying it out.